# Basic implementation of MCTS to Classifier Chains

## Loss comparison

In [ ]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split

n_samples = 10000
n_features = 64
n_classes = 32
n_labels = 2
random_state = 0

X, Y = make_multilabel_classification(
    n_samples = n_samples,
    n_features = n_features,
    n_classes = n_classes,
    n_labels = n_labels,
    random_state = random_state)

test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)

from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression

solver = "liblinear"
base = LogisticRegression(solver=solver)
chain = ClassifierChain(base)

chain = chain.fit(X_train, Y_train)

from src.mcts_inference.mcts import MCTS
from src.mcts_inference.mcts_config import MCTSConfig
from src.mcts_inference.constraints import Constraint
from src.mcts_inference.policy import UCB 

constraint = Constraint(max_iter=True, n_iter=1000)
config1 = MCTSConfig(n_classes, selection_policy=UCB(), constraint=constraint, verbose=True, step_once=True)
config2 = MCTSConfig(n_classes                        , constraint=constraint, verbose=True, step_once=True)

M: int = 100
out1 = MCTS(chain, X_test[:M], config1)
out2 = MCTS(chain, X_test[:M], config2)

from sklearn.metrics import hamming_loss

print("Hamming loss of chain              : ", hamming_loss(chain.predict(X_test[:M]), Y_test[:M]))
print("Hamming loss of MCTS UCB           : ", hamming_loss(out1, Y_test[:M]))
print("Hamming loss of MCTS Epsilon Greedy: ", hamming_loss(out2, Y_test[:M]))

# Implementation seems fine, now let's compare accuracies

In [ ]:
from sklearn.metrics import hamming_loss, zero_one_loss
from tqdm import trange

secs_lis = [0.1, 0.5, 1.] #  2.]
M = min(100,len(Y_test))

hl_mt = []
hl_ct = []
hl_mc = []

zo_mt = []
zo_ct = []
zo_mc = []

y_chain = chain.predict(X_test[:M])
for secs in secs_lis:
#     continue
    y_mcts = []

    for i in trange(M, desc=f"MCTS Inference Constraint={secs}s", unit="it",colour="green"):
        y_mcts.append(MCTS(chain,X_test[i],secs=secs))

    y_mcts = np.array(y_mcts)
    
    hl_mt.append(hamming_loss(y_mcts,Y_test[:M]))
    hl_ct.append(hamming_loss(y_chain,Y_test[:M]))
    hl_mc.append(hamming_loss(y_chain,y_mcts))

    zo_mt.append(zero_one_loss(y_mcts,Y_test[:M]))
    zo_ct.append(zero_one_loss(y_chain,Y_test[:M]))
    zo_mc.append(zero_one_loss(y_chain,y_mcts))

# Plot
import matplotlib.pyplot as plt

plt.plot(secs_lis,hl_mt,label="MCTS vs True")
plt.plot(secs_lis,hl_ct,label="Chains vs True")
plt.plot(secs_lis,hl_mc,label="MCTS vs Chains")

plt.title("Hamming Loss Comparison for different times")
plt.xlabel("Seconds")
plt.ylim(0,1)
plt.ylabel("Hamming Loss")
plt.legend()
plt.show();

plt.plot(secs_lis,zo_mt,label="MCTS vs True")
plt.plot(secs_lis,zo_ct,label="Chains vs True")
plt.plot(secs_lis,zo_mc,label="MCTS vs Chains")

plt.title("Zero One Loss Comparison for time different times")
plt.xlabel("Seconds")
plt.ylim(0,1)
plt.ylabel("Zero One Loss")
plt.legend()
plt.show();

## Iteration to time

In [ ]:
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split

from src.mcts_inference.mcts import MCTS
from src.mcts_inference.mcts_config import MCTSConfig
from src.mcts_inference.constraints import Constraint
from src.mcts_inference.policy import UCB 

from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
solver = "liblinear"

import time

Ns = [100, 500, 1000, 1500, 2000]
Ls = [2, 4, 8, 16, 32, 64]
avg = 10
M = 100

n_samples = 10000
n_labels = 2
random_state = 0
test_size = 0.2

In [ ]:
times = [[[] for _ in Ns] for _ in Ls]

for il, l in enumerate(Ls):
    t = time.time()
    print("L>", l)
    X, Y = make_multilabel_classification(
        n_samples = n_samples,
        n_features = 4 * l,
        n_classes = l,
        n_labels = n_labels,
        random_state = random_state)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)

    base = LogisticRegression(solver=solver)
    chain = ClassifierChain(base)

    chain = chain.fit(X_train,Y_train)
    
    print(">n:", end=" ")
    for jn, n in enumerate(Ns):
        tt = time.time()
        print(n, end=", ")
        constraint = Constraint(max_iter=True, n_iter=n)
        config = MCTSConfig(l, selection_policy=UCB(), constraint=constraint, verbose=False, step_once=False, parallel=True)
        for _ in range(avg):
            t0 = time.time()

            MCTS(chain, X_test[:M], config)

            times[il][jn].append(time.time()-t0)
        print(f"({time.time()-tt:.2f}s)", end = " ")
    print(f"\n({time.time()-t:.3f}s)")
times

In [ ]:
np.array(np.mean(times,axis=-1))  # Store z, took 40 minutes to run.
# array([[ 0.07813816,  0.16268473,  0.26474874,  0.36404204,  0.51120052],
#        [ 0.11869841,  0.3139014 ,  0.534267  ,  0.74213264,  0.95371227],
#        [ 0.41399403,  1.13965526,  1.56720562,  2.00165966,  2.41907544],
#        [ 0.88097756,  3.77394373,  7.51101863, 11.32408068, 15.06112485],
#        [ 1.58284893,  7.38303893, 14.78826919, 22.28634632, 29.75484819],
#        [ 3.07422526, 14.72650027, 29.62631023, 44.4082509 , 59.50828764]])

In [ ]:
for l in Ls:
    t = time.time()
    print("L>", l)
    print("  n:", end=" ")
    for n in Ns:
        tt = time.time()
        print(n, end=", ")
        time.sleep(0.2)
        print(f"({time.time()-tt:.2f}s)", end=" ")
    print(f"({time.time()-t:.3f}s)")    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import cbook, cm
from matplotlib.colors import LightSource


x, y = np.meshgrid(Ns, Ls)
z = np.array(np.mean(times,axis=-1))

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_wireframe(x, y, z, rstride=1, cstride=1, linewidth=1, alpha=1., label="Time")
surf = ax.plot_wireframe(x, y, np.log(z), rstride=1, cstride=1, linewidth=1, alpha=0.5, label="Log Time")


plt.xticks(Ns, fontsize=8)
plt.yticks(Ls, fontsize=8)
ax.set_xlabel('Number of Iterations',fontsize=10)
ax.set_ylabel('Number of Labels',fontsize=10)
ax.set_zlabel('Time (s)',fontdict={'fontsize': 10})

fig.colorbar(surf,shrink=0., aspect=1)

plt.legend(loc=6)
plt.show()

In [ ]:
for zz, ll in zip(z, Ls):
    plt.plot(Ns,zz,marker="o",label=f"L={ll}",markersize=2)
plt.xticks(Ns)
plt.legend()
plt.title("Time (s) vs Number of Iterations")
plt.show();

### Obtain the specs of the machine.

In [ ]:
# number of cpu cores
import multiprocessing
print(f"CPU count: {multiprocessing.cpu_count()}\n")
!lscpu